## Notebook to test Optimal controller on MS

### Notes

- Change picked peak file to whatever is most up-to-date
- Change MS from Simulator to real :-)
- Fix paths

In [1]:
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import xml.etree.ElementTree
import os,glob,sys
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
sys.path.append('/home/simon/git/vimms')
sys.path.append('/home/simon/git/pymzm')

## Change following to where results should go...

In [4]:
test_path = '/home/simon/git/vimms/experimental/simon_res/'

## Change following to where the picked peaks file is

In [15]:
pp_file = os.path.join(test_path,'QCB','qcb_pp.csv')

In [5]:
from vimms.PythonMzmine import *
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import TopNController,RoiController,SmartRoiController
from vimms.PythonMzmine import *

from vimms.Roi import make_roi, RoiToChemicalCreator
from vimms.BOMAS import *
from vimms.Common import *
from vimms.Environment import *
from pathlib import Path
from vimms.PlotsForPaper import get_frag_events

In [6]:
set_log_level_warning()

Following not needed on real machine, just for simulation...

In [8]:
# base_dir = os.path.abspath('/Users/simon/University of Glasgow/Vinny Davies - CLDS Metabolomics Project/Trained Models')
base_dir = os.path.abspath('/home/simon/git/vimms/experimental/simon_res/QCB')

ps_frag_QCB = load_obj(Path(base_dir, 'peak_sampler_mz_rt_int_beerqcb_fragmentation.p'))

QCB_MZML2CHEMS_DICT = {'min_ms1_intensity': 0,
                  'mz_tol': 5,
                  'mz_units':'ppm',
                  'min_length':1,
                  'min_intensity':0,
                  'start_rt':0,
                  'stop_rt':1560}

TopN_QCB_dataset = load_obj('/home/simon/git/vimms/experimental/simon_res/QCB/TopN_QCB_dataset.mzml')

Not all of this needed, but no hassle to do it...

In [23]:
min_rt = 0
max_rt = 26*60 # entire run
min_ms1_intensity = 5000
mz_tol = 10
rt_tol = 15
N = 10

# these are derived from real data (see bottom of notebook)
roi_time_dict = {1: 0.71,2:0.20}
topn_time_dict = {1: 0.60,2:0.20}


ionisation_mode = POSITIVE
isolation_width = 1

## Following defines how to prioritise boxes - suggest leaving intensity for now. If we have time to try two, also try "apex"

In [24]:
score_method = 'intensity'

In [25]:
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import OptimalTopNController

## Following cells show non-looped version

In [26]:
boxes = load_picked_boxes(pp_file)

In [27]:
controller = OptimalTopNController(ionisation_mode, N, isolation_width,
                                   mz_tol, rt_tol, min_ms1_intensity,
                                   boxes,score_method = score_method)

In [28]:
mass_spec = IndependentMassSpectrometer(ionisation_mode, TopN_QCB_dataset,
                                        ps_frag_QCB,
                                        add_noise=True,
                                       scan_duration_dict=roi_time_dict)
env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)
env.run()

(100.200s) ms_level=1 N=10 DEW=15:  99%|█████████▉| 99.48999999999977/100 [00:04<00:00, 24.64it/s]


## And here is the looped version

Main points:

- Load the list of box objects once
- They get removed once fragmented
- Making a new controller with the same list will only use the remaining ones

I guess on the machine you will need to embed this within the sequencer.

In [29]:
boxes = load_picked_boxes(pp_file) # load boxes once

In [32]:
print(len(boxes)) # initial number
for rep in range(3): # or whatever
    controller = OptimalTopNController(ionisation_mode, N, isolation_width,
                                   mz_tol, rt_tol, min_ms1_intensity,
                                   boxes,score_method = score_method)
    mass_spec = IndependentMassSpectrometer(ionisation_mode, TopN_QCB_dataset,
                                        ps_frag_QCB,
                                        add_noise=True,
                                       scan_duration_dict=roi_time_dict)
    env = Environment(mass_spec, controller, min_rt, max_rt, progress_bar=True)
    env.run()
    out_file = 'optimal_{}_qcb_{}.mzml'.format(score_method,rep)
    env.write_mzML(test_path,out_file)
    print(len(boxes))

5879


(300.020s) ms_level=1 N=10 DEW=15:  99%|█████████▉| 99.31000000000026/100 [00:06<00:00, 16.48it/s] 


5599


(300.580s) ms_level=1 N=10 DEW=15: 100%|█████████▉| 99.86999999999989/100 [00:05<00:00, 17.90it/s] 


5512


KeyboardInterrupt: 